In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.utils.data as data
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
from torch.utils.data import  Dataset

In [3]:
import tensorflow as tf

In [4]:
DATA_DIR = 'gs://time_series_datasets'
LOCAL_CACHE_DIR = './data_loader/dataset/'

In [5]:
data="weather"

In [6]:
feature_type='MS'
target='OT'

In [7]:
target_slice=slice(0, None)

In [44]:
seq_len=int(336)
pred_len=int(96)

In [9]:
if not os.path.isdir(LOCAL_CACHE_DIR):
    os.mkdir(LOCAL_CACHE_DIR)
file_name = data + '.csv'
cache_filepath = os.path.join(LOCAL_CACHE_DIR, file_name)
if not os.path.isfile(cache_filepath):
    tf.io.gfile.copy(
  os.path.join(DATA_DIR, file_name), cache_filepath, overwrite=True
  )
    # Download the data from the cloud storage
    # (Implement cloud storage download here)
df_raw = pd.read_csv(cache_filepath)
df = df_raw.set_index('date')

In [10]:
df.shape

(52696, 21)

In [11]:
if feature_type == 'S':
    df = df[[target]]
elif feature_type == 'MS':
    target_idx = df.columns.get_loc(target)
    target_slice = slice(target_idx, target_idx + 1)
# split train/valid/test
n = len(df)

In [12]:
target_slice 

slice(20, 21, None)

In [13]:
if data.startswith('ETTm'):
    train_end = 12 * 30 * 24 * 4
    val_end = train_end + 4 * 30 * 24 * 4
    test_end = val_end + 4 * 30 * 24 * 4
elif data.startswith('ETTh'):
     train_end = 12 * 30 * 24
     val_end = train_end + 4 * 30 * 24
     test_end = val_end + 4 * 30 * 24
else:
    train_end = int(n * 0.7)
    val_end = n - int(n * 0.2)
    test_end = n

In [14]:
train_end, val_end , test_end 

(36887, 42157, 52696)

In [15]:
train_df = df[:train_end]
val_df = df[train_end - seq_len : val_end]
test_df = df[val_end - seq_len : test_end]

In [71]:
train_df.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),...,wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m�),PAR (�mol/m�/s),max. PAR (�mol/m�/s),Tlog (degC),OT
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:10:00,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,3.42,5.49,...,1.02,1.60,224.3,0.0,0.0,0.0,0.0,0.0,11.45,428.1
2020-01-01 00:20:00,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,3.39,5.45,...,0.43,0.84,206.8,0.0,0.0,0.0,0.0,0.0,11.51,428.0
2020-01-01 00:30:00,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,3.39,5.43,...,0.61,1.48,197.1,0.0,0.0,0.0,0.0,0.0,11.60,427.6
2020-01-01 00:40:00,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,3.35,5.37,...,1.11,1.48,206.4,0.0,0.0,0.0,0.0,0.0,11.70,430.0
2020-01-01 00:50:00,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,3.38,5.42,...,0.49,1.40,209.6,0.0,0.0,0.0,0.0,0.0,11.81,432.2


In [23]:
scaler = StandardScaler()
scaler.fit(train_df.values)

StandardScaler()

In [24]:
def scale_df(df, scaler):
    data = scaler.transform(df.values)
    return pd.DataFrame(data, index=df.index, columns=df.columns)

In [25]:
train_df = scale_df(train_df, scaler)
val_df = scale_df(val_df, scaler)
test_df = scale_df(test_df, scaler)
n_feature = train_df.shape[-1]

In [63]:
def _split_window(data):
    inputs = data[:, :seq_len]
    labels = data[:, seq_len: ]
    inputs.set_shape([None, seq_len, None])
    labels.set_shape([None, pred_len, None])
    return inputs, labels

In [33]:
def _make_dataloader(data, shuffle=True):
    inputs, labels =_split_window(data)
    input_data = np.array(inputs, dtype=np.float32)
    input_dataset = torch.tensor(input_data)
    label_data = np.array(labels, dtype=np.float32)
    label_dataset = torch.tensor(label_data)
    input = DataLoader(input_dataset, batch_size=self.batch_size, shuffle=shuffle)
    label = DataLoader(label_dataset , batch_size=self.batch_size, shuffle=shuffle)
    #return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)
    return input , label

In [ ]:
def __len__():
    return len(data) - (seq_len + pred_len) + 1

def __getitem__(idx):
    idx += seq_len  # Adjust index to consider the sequence length
    data_window = data[idx - seq_len: idx + pred_len]
    inputs = data_window[:seq_len, :]
    labels = data_window[seq_len:, target_slice]
    return torch.tensor(inputs, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)

In [36]:
class TSDataset(Dataset):
    def __init__(self, data, seq_len, pred_len, feature_type, target='OT'):
        self.data = 'weather'
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.feature_type = feature_type
        self.target = target
        self.target_slice = slice(0, None)
        self._read_data()

    def _read_data(self):
        if not os.path.isdir(LOCAL_CACHE_DIR):
            os.mkdir(LOCAL_CACHE_DIR)
        file_name = f'{self.data}.csv'
        cache_filepath = os.path.join(LOCAL_CACHE_DIR, file_name)
        if not os.path.isfile(cache_filepath):
            # Copy data from cloud storage if not exists locally
            pass  # Add code for copying data from DATA_DIR to LOCAL_CACHE_DIR here

        df_raw = pd.read_csv(cache_filepath)
        df = df_raw.set_index('date')
        if self.feature_type == 'S':
            df = df[[self.target]]
        elif self.feature_type == 'MS':
            target_idx = df.columns.get_loc(self.target)
            self.target_slice = slice(target_idx, target_idx + 1)

        n = len(df)
        if self.data.startswith('ETTm'):
            train_end = 12 * 30 * 24 * 4
            val_end = train_end + 4 * 30 * 24 * 4
            test_end = val_end + 4 * 30 * 24 * 4
        elif self.data.startswith('ETTh'):
            train_end = 12 * 30 * 24
            val_end = train_end + 4 * 30 * 24
            test_end = val_end + 4 * 30 * 24
        else:
            train_end = int(n * 0.7)
            val_end = n - int(n * 0.2)
            test_end = n
        train_df = df[:train_end]
        val_df = df[train_end - self.seq_len: val_end]
        test_df = df[val_end - self.seq_len: test_end]

        self.scaler = StandardScaler()
        self.scaler.fit(train_df.values)

        self.train_data = self.scale_df(train_df)
        self.val_data = self.scale_df(val_df)
        self.test_data = self.scale_df(test_df)
        self.n_feature = self.train_data.shape[-1]

        return self.train_data, self.val_data, self.test_data, self.n_feature 

    def scale_df(self, df):
        data = self.scaler.transform(df.values)
        return pd.DataFrame(data, index=df.index, columns=df.columns)

    def __len__(self):
        return len(self.data) - (self.seq_len + self.pred_len) + 1

    def __getitem__(self, idx):
        idx += self.seq_len  # Adjust index to consider the sequence length
        data_window = self.data[idx - self.seq_len: idx + self.pred_len]
        inputs = data_window[:self.seq_len, :]
        labels = data_window[self.seq_len:, self.target_slice]
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)


In [59]:
class TSDataModule(pl.LightningDataModule):
    def __init__(self, data, batch_size, seq_len, pred_len, feature_type, target='OT'):
        super(TSDataModule, self).__init__()
        self.data = data
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.feature_type = feature_type
        self.target = target

    def setup(self, stage=None):
        self.train_dataset, self.val_dataset, self.test_data, self.n_feature = TSDataset(self.data, self.seq_len, self.pred_len, self.feature_type, self.target)._read_data()
        #self.val_dataset = TSDataset(self.data.val_df.values, self.seq_len, self.pred_len, self.feature_type, self.target)
        #self.test_dataset = TSDataset(self.data.test_df.values, self.seq_len, self.pred_len, self.feature_type, self.target)
        #self.n_feature = self.train_dataset.n_feature

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

In [38]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split
import pytorch_lightning as pl
import torchmetrics
from torchmetrics import Metric


class MyAccuracy(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("correct", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds, target):
        preds = torch.argmax(preds, dim=1)
        assert preds.shape == target.shape
        self.correct += torch.sum(preds == target)
        self.total += target.numel()

    def compute(self):
        return self.correct.float() / self.total.float()


class NN(pl.LightningModule):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(
            task="multiclass", num_classes=num_classes
        )
        self.my_accuracy = MyAccuracy()
        self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        accuracy = self.my_accuracy(scores, y)
        f1_score = self.f1_score(scores, y)
        self.log_dict(
            {
                "train_loss": loss,
                "train_accuracy": accuracy,
                "train_f1_score": f1_score,
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return {"loss": loss, "scores": scores, "y": y}

    def validation_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("test_loss", loss)
        return loss

    def _common_step(self, batch, batch_idx):
        x, y = batch
        x = x.reshape(x.size(0), -1)
        scores = self.forward(x)
        loss = self.loss_fn(scores, y)
        return loss, scores, y

    def predict_step(self, batch, batch_idx):
        x, y = batch
        x = x.reshape(x.size(0), -1)
        scores = self.forward(x)
        preds = torch.argmax(scores, dim=1)
        return preds

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

In [39]:
# Hyperparameters
input_size = 21
num_classes = 2
learning_rate = 0.001
batch_size = 64
num_epochs = 3

In [60]:
# Instantiate the TSDataModule
data_module = TSDataModule(data='weather',  # Replace with your actual data object
                           batch_size=32,
                           seq_len=int(336),
                           pred_len=int(96),
                           feature_type='MS',  # Replace with 'S' or 'MS' based on your data
                           target='OT')  # Replace with your target feature name

In [61]:
td = data_module.train_dataloader()

AttributeError: 'TSDataModule' object has no attribute 'train_dataset'

In [53]:
model = NN(input_size=input_size, num_classes=num_classes)

In [54]:
trainer = pl.Trainer(
    accelerator="gpu", devices=1, min_epochs=1, max_epochs=3, precision=16
)

/Users/abidakunabisoye/deep_learning/pytorch_lightning/venv/lib/python3.9/site-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
/Users/abidakunabisoye/deep_learning/pytorch_lightning/venv/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:125: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [57]:
trainer.fit(model, datamodule=data_module)
trainer.validate(model, datamodule=data_module)
trainer.test(model, datamodule=data_module)

TypeError: string indices must be integers

In [22]:


# Test the model
trainer.test(model, datamodule=data_module)

# Use the trained model for predictions
# Example: Get test data loader and make predictions
test_dataloader = data_module.test_dataloader()
predictions = []
for batch in test_dataloader:
    inputs, _ = batch
    predictions_batch = model(inputs)
    predictions.append(predictions_batch.detach().cpu().numpy())

# Concatenate predictions into a numpy array
predictions = np.concatenate(predictions, axis=0)
print(predictions)

TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `YourModel`

In [62]:
import tensorflow as geek 
  
  
x = geek.constant([[[ 1, 2, 3], 
                    [ 4, 5, 6]], 
                   [[ 7, 8, 9], 
                    [ 10, 11, 12]], 
                   [[ 13, 14, 15], 
                    [ 16, 17, 18]], 
                   [[ 19, 20, 21], 
                    [ 22, 23, 24]]]) 

In [63]:
x.shape

TensorShape([4, 2, 3])

In [64]:
transposed_tensor = geek.transpose(x, perm = [0, 2, 1])

In [67]:
transposed_tensor.shape, transposed_tensor.shape[-1]

(TensorShape([4, 3, 2]), 2)

In [68]:
transposed_tensor

<tf.Tensor: shape=(4, 3, 2), dtype=int32, numpy=
array([[[ 1,  4],
        [ 2,  5],
        [ 3,  6]],

       [[ 7, 10],
        [ 8, 11],
        [ 9, 12]],

       [[13, 16],
        [14, 17],
        [15, 18]],

       [[19, 22],
        [20, 23],
        [21, 24]]], dtype=int32)>

In [72]:
x = np.random.rand(2, 3, 10)
print("Original Input Tensor:")
print(x)

Original Input Tensor:
[[[0.47514888 0.47613429 0.93765199 0.79252459 0.63574814 0.12671022
   0.48848839 0.98890659 0.19326399 0.66808064]
  [0.15818084 0.28068655 0.32338139 0.82462328 0.85483169 0.18806031
   0.79114297 0.36286764 0.17001837 0.86611145]
  [0.48137374 0.06788061 0.01775989 0.47398271 0.78528897 0.1472132
   0.11111811 0.88943838 0.41352883 0.63053788]]

 [[0.91967109 0.36178371 0.6083693  0.43405874 0.74289473 0.74715923
   0.64159177 0.79346753 0.41659555 0.50756065]
  [0.58041969 0.57648182 0.80107985 0.30507917 0.74441683 0.04015197
   0.3019863  0.30614458 0.32898332 0.71063726]
  [0.76552368 0.11061252 0.02050674 0.79833809 0.02673619 0.76268763
   0.69708581 0.51955096 0.92430361 0.48559953]]]


In [73]:
# Transpose the input tensor for processing in the model
x_transposed = np.transpose(x, (0, 2, 1))  # [Batch, Input Length, Channel]
print("\nTransposed Input Tensor:")
print(x_transposed)


Transposed Input Tensor:
[[[0.47514888 0.15818084 0.48137374]
  [0.47613429 0.28068655 0.06788061]
  [0.93765199 0.32338139 0.01775989]
  [0.79252459 0.82462328 0.47398271]
  [0.63574814 0.85483169 0.78528897]
  [0.12671022 0.18806031 0.1472132 ]
  [0.48848839 0.79114297 0.11111811]
  [0.98890659 0.36286764 0.88943838]
  [0.19326399 0.17001837 0.41352883]
  [0.66808064 0.86611145 0.63053788]]

 [[0.91967109 0.58041969 0.76552368]
  [0.36178371 0.57648182 0.11061252]
  [0.6083693  0.80107985 0.02050674]
  [0.43405874 0.30507917 0.79833809]
  [0.74289473 0.74441683 0.02673619]
  [0.74715923 0.04015197 0.76268763]
  [0.64159177 0.3019863  0.69708581]
  [0.79346753 0.30614458 0.51955096]
  [0.41659555 0.32898332 0.92430361]
  [0.50756065 0.71063726 0.48559953]]]


In [74]:
# Apply target slicing to select channels 1 and 2
target_slice = slice(1, 3)

In [75]:
target_slice 

slice(1, 3, None)

In [76]:
sliced_output = x_transposed[:, :, target_slice]

In [77]:
print("\nSliced Output Tensor (Channels 1 and 2):")
print(sliced_output)


Sliced Output Tensor (Channels 1 and 2):
[[[0.15818084 0.48137374]
  [0.28068655 0.06788061]
  [0.32338139 0.01775989]
  [0.82462328 0.47398271]
  [0.85483169 0.78528897]
  [0.18806031 0.1472132 ]
  [0.79114297 0.11111811]
  [0.36286764 0.88943838]
  [0.17001837 0.41352883]
  [0.86611145 0.63053788]]

 [[0.58041969 0.76552368]
  [0.57648182 0.11061252]
  [0.80107985 0.02050674]
  [0.30507917 0.79833809]
  [0.74441683 0.02673619]
  [0.04015197 0.76268763]
  [0.3019863  0.69708581]
  [0.30614458 0.51955096]
  [0.32898332 0.92430361]
  [0.71063726 0.48559953]]]


In [78]:
print(x_transposed)

[[[0.47514888 0.15818084 0.48137374]
  [0.47613429 0.28068655 0.06788061]
  [0.93765199 0.32338139 0.01775989]
  [0.79252459 0.82462328 0.47398271]
  [0.63574814 0.85483169 0.78528897]
  [0.12671022 0.18806031 0.1472132 ]
  [0.48848839 0.79114297 0.11111811]
  [0.98890659 0.36286764 0.88943838]
  [0.19326399 0.17001837 0.41352883]
  [0.66808064 0.86611145 0.63053788]]

 [[0.91967109 0.58041969 0.76552368]
  [0.36178371 0.57648182 0.11061252]
  [0.6083693  0.80107985 0.02050674]
  [0.43405874 0.30507917 0.79833809]
  [0.74289473 0.74441683 0.02673619]
  [0.74715923 0.04015197 0.76268763]
  [0.64159177 0.3019863  0.69708581]
  [0.79346753 0.30614458 0.51955096]
  [0.41659555 0.32898332 0.92430361]
  [0.50756065 0.71063726 0.48559953]]]


In [79]:
sliced_output_transposed = np.transpose(sliced_output, (0, 2, 1)) 

In [80]:
print(sliced_output_transposed)

[[[0.15818084 0.28068655 0.32338139 0.82462328 0.85483169 0.18806031
   0.79114297 0.36286764 0.17001837 0.86611145]
  [0.48137374 0.06788061 0.01775989 0.47398271 0.78528897 0.1472132
   0.11111811 0.88943838 0.41352883 0.63053788]]

 [[0.58041969 0.57648182 0.80107985 0.30507917 0.74441683 0.04015197
   0.3019863  0.30614458 0.32898332 0.71063726]
  [0.76552368 0.11061252 0.02050674 0.79833809 0.02673619 0.76268763
   0.69708581 0.51955096 0.92430361 0.48559953]]]
